In [5]:
print('Multi_Agent_Rag')

Multi_Agent_Rag


# PDF

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
os.environ["OPENAI_API_KEY"]=os.getenv("openai_api_key")
os.environ["LANGCHAIN_TRACING_V2"]="True"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("langsmith_api_key")

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [27]:
pdf_files = [
    "attention_is_all_you_need.pdf",
    "OOPS_Interview_Cheatsheet.pdf",
    "DBMS_my_notes.pdf",
    "OS_Full_Notes.pdf",
    "DBMS_my_notes.pdf",
    "SQL_notes.pdf"
]

In [28]:
for pdf_file in pdf_files:

    loader=PyPDFLoader(pdf_file)
    documents=loader.load()
    
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    split_documents=text_splitter.split_documents(documents)
    
    db_all=Chroma.from_documents(split_documents,OpenAIEmbeddings())

Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 93 0 (offset 0)
Ignoring wrong pointing object 135 0 (offset 0)
Ignoring wrong pointing object 138 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 204 0 (offset 0)
Ignoring wrong pointing object 206 0 (offset 0)
Ignoring wrong pointing object 210 0 (offset 0)
Ignoring wrong pointing object 223 0 (offset 0)
Ignoring wrong pointing object 234 0 (offset 0)
Ignoring wrong pointing object 237 0 (offset 0)
Ignoring wrong pointing object 241 0 (offset 0)
Ignoring wrong pointing object 260 0 (offset 0)
Ignoring wrong pointing object 262 0 (offset 0)
Ignoring wrong pointing object 264 0 (offset 0)
Ignoring wrong pointing object 269 0 (offset 0)

In [29]:
'''
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("attention_is_all_you_need.pdf")
Attention=loader.load()
'''

'\nfrom langchain_community.document_loaders import PyPDFLoader\nloader=PyPDFLoader("attention_is_all_you_need.pdf")\nAttention=loader.load()\n'

In [30]:
'''
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=200)
Attention_split=text_splitter.split_documents(Attention)
'''

'\nfrom langchain_text_splitters import RecursiveCharacterTextSplitter\ntext_splitter=RecursiveCharacterTextSplitter(chunk_size=800,chunk_overlap=200)\nAttention_split=text_splitter.split_documents(Attention)\n'

In [31]:
'''
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
'''

'\nfrom langchain_openai import OpenAIEmbeddings\nfrom langchain_community.vectorstores import Chroma\n'

In [32]:
'''
Attention_db=Chroma.from_documents(Attention_split,OpenAIEmbeddings())
'''

'\nAttention_db=Chroma.from_documents(Attention_split,OpenAIEmbeddings())\n'

In [33]:
pdf_retriever=db_all.as_retriever()
pdf_retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F1D700FD70>, search_kwargs={})

In [34]:
from langchain.tools.retriever import create_retriever_tool
pdf_retriever_tool=create_retriever_tool(pdf_retriever,"Pdf_search","A tool to search for and extract information from PDF documents")

In [35]:
tools=[pdf_retriever_tool]
tools

[Tool(name='Pdf_search', description='A tool to search for and extract information from PDF documents', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001F1CE8EE8E0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F1D700FD70>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001F1CEA704A0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001F1D700FD70>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_sepa

In [36]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4-turbo",temperature=0)

In [37]:
from langchain import hub
Prompt=hub.pull("hwchase17/openai-functions-agent")
Prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Agents

In [38]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,Prompt)

In [39]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=False)
agent_executor

AgentExecutor(verbose=False, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Ta

In [40]:
agent_executor.invoke({"input":"Who are authors of paper Attention is all you need ?"})

{'input': 'Who are authors of paper Attention is all you need ?',
 'output': 'The authors of the paper "Attention is All You Need" are Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, and Illia Polosukhin.'}

In [41]:
agent_executor.invoke({"input":"Tell me two things 1)Who are authors of paper Attention is all you need ? and 2) What is attention ?"})

{'input': 'Tell me two things 1)Who are authors of paper Attention is all you need ? and 2) What is attention ?',
 'output': '### 1) Authors of the paper "Attention is All You Need"\nThe authors of the paper "Attention is All You Need" are:\n- Ashish Vaswani\n- Noam Shazeer\n- Niki Parmar\n- Jakob Uszkoreit\n- Llion Jones\n- Aidan N. Gomez\n- Łukasz Kaiser\n- Illia Polosukhin\n\n### 2) What is Attention?\nThe concept of "attention" in the context of the paper "Attention is All You Need" refers to a mechanism in neural networks that allows the model to focus on different parts of the input sequence when predicting parts of the output sequence. This mechanism is crucial for tasks such as translation, where the relevance of input elements can vary depending on the context. The attention mechanism helps the model to dynamically weigh the importance of input elements and is a key component of the Transformer architecture introduced in the paper.'}

In [42]:
agent_executor.invoke({"input":"Answer me various questions 1)Who are authors of paper Attention is all you need ? and 2) What is const keyword ? and 3) What is master slave database concept ? 4) Expalin difference between 32 bit and 64 bit OS ? 5)What are co-realted sub queries ?"})

{'input': 'Answer me various questions 1)Who are authors of paper Attention is all you need ? and 2) What is const keyword ? and 3) What is master slave database concept ? 4) Expalin difference between 32 bit and 64 bit OS ? 5)What are co-realted sub queries ?',
 'output': 'Here are the answers to your questions:\n\n1) **Authors of the paper "Attention is All You Need"**:\n   - Ashish Vaswani\n   - Noam Shazeer\n   - Niki Parmar\n   - Jakob Uszkoreit\n   - Llion Jones\n   - Aidan N. Gomez\n   - Łukasz Kaiser\n   - Illia Polosukhin\n\n2) **What is the `const` keyword?**\n   The `const` keyword in programming languages like C, C++, and JavaScript is used to declare variables that cannot be reassigned after their initial assignment. This means that the value of a `const` variable must remain constant throughout the scope in which it is declared. In C and C++, `const` can also be used to make function parameters and object methods promise not to modify the data they are called with.\n\n3) 

In [44]:
agent_executor.invoke({"input":"Answer me various questions in short answers only max one line per question 1)Is Abhi Arora authors of paper Attention is all you need ? and 2) Are joins faster than subqueries ? and 3) Which keyword is used to modify a methond,property and indexer or event declaration ? 4)How many unique memory address a 64 bit register can hold ? 5) Who is Varun ?"})

{'input': 'Answer me various questions in short answers only max one line per question 1)Is Abhi Arora authors of paper Attention is all you need ? and 2) Are joins faster than subqueries ? and 3) Which keyword is used to modify a methond,property and indexer or event declaration ? 4)How many unique memory address a 64 bit register can hold ? 5) Who is Varun ?',
 'output': '1) No, Abhi Arora is not an author of the paper "Attention is All You Need"; the authors include Ashish Vaswani, Noam Shazeer, and others.\n2) Joins are generally faster than subqueries as they maximize calculation burden on the DBMS.\n3) The keyword used to modify a method, property, indexer, or event declaration is not specified in the provided data.\n4) A 64-bit register can hold 2^64 unique memory addresses.\n5) Information about "Who is Varun?" is not provided in the data.'}

# Excel

In [84]:
import nest_asyncio
nest_asyncio.apply()

In [85]:
from llama_parse import LlamaParse
llama_key=os.getenv("llama_api_key")

In [86]:
parser=LlamaParse(api_key=llama_key,result_type="markdown")

In [87]:
documents=parser.load_data("pokemon_data.csv")

Started parsing the file under job_id c631e86b-5f54-4aa8-a0ae-f76ea75b825b


In [88]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

In [89]:
gpt=OpenAI(model="gpt-4-turbo")
Settings.llm=gpt

In [90]:
from llama_index.core import VectorStoreIndex
index=VectorStoreIndex.from_documents(documents)

In [91]:
query_engine=index.as_query_engine()

In [92]:
response=query_engine.query("Difference between KyuremBlack Kyurem and KyuremWhite Kyurem ?")
str(response)

"Kyurem Black Kyurem and Kyurem White Kyurem differ primarily in their attack and special attack stats. Kyurem Black Kyurem has a higher attack stat at 170 compared to Kyurem White Kyurem's 120, while Kyurem White Kyurem has a higher special attack stat at 170 compared to Kyurem Black Kyurem's 120. Both forms have the same HP, defense, special defense, and speed stats."

In [93]:
response=query_engine.query("Tell me two things 1)What are stats of Ivysaur and 2)Who is stronger Lugia or Magmar and also their stats ? ?")
str(response)

"1) The stats for Ivysaur are as follows: HP: 60, Attack: 62, Defense: 63, Special Attack: 80, Special Defense: 80, Speed: 60.\n\n2) Comparing Lugia and Magmar:\n   - Lugia's stats are: HP: 106, Attack: 90, Defense: 130, Special Attack: 90, Special Defense: 154, Speed: 110.\n   - Magmar's stats are: HP: 65, Attack: 95, Defense: 57, Special Attack: 100, Special Defense: 85, Speed: 93.\n\nLugia is stronger than Magmar based on overall higher stats in HP, Defense, Special Defense, and Speed."

In [97]:
response=query_engine.query("What are stats of pokemon Ivysaur?")
str(response)

'Ivysaur has the following stats:\n- HP: 60\n- Attack: 62\n- Defense: 63\n- Special Attack: 80\n- Special Defense: 80\n- Speed: 60'

In [98]:
response=query_engine.query("What are stats of Starmie?")
str(response)

'Starmie has the following stats: HP 60, Attack 75, Defense 85, Special Attack 100, Special Defense 85, Speed 115.'

In [100]:
response=query_engine.query("What are stats of Golurk?")
str(response)

'Golurk has the following stats:\n- HP: 89\n- Attack: 124\n- Defense: 80\n- Special Attack: 55\n- Special Defense: 80\n- Speed: 55'

In [62]:
from langchain.tools import Tool

def excel_search_tool(input_text:str):
    response=query_engine.query(input_text)
    return str(response)

excel_retriever_tool=Tool(
    name="Excel_search",
    func=excel_search_tool,
    description="A tool to search for and extract information from Excel documents"
)

tools=[pdf_retriever_tool,excel_retriever_tool]
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [65]:
response=agent_executor.invoke({"input":"Tell me two things 1)What are stats of Ivysaur and 2)Who is stronger Lugia or Magmar and also their stats ? ?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Pdf_search` with `{'query': 'Ivysaur stats'}`


We trained a 4-layer transformer with dmodel = 1024 on the Wall Street Journal (WSJ) portion of the
Penn Treebank [ 25], about 40K training sentences. We also trained it in a semi-supervised setting,
using the larger high-confidence and BerkleyParser corpora from with approximately 17M sentences
[37]. We used a vocabulary of 16K tokens for the WSJ only setting and a vocabulary of 32K tokens
for the semi-supervised setting.
We performed only a small number of experiments to select the dropout, both attention and residual
(section 5.4), learning rates and beam size on the Section 22 development set, all other parameters
remained unchanged from the English-to-German base translation model. During inference, we
9

We trained a 4-layer transformer with dmodel = 1024 on the Wall Street Journal (WSJ) portion of the
Penn Treebank [ 25], about 40K training sentences. We also trained it in a semi-

In [67]:
response=agent_executor.invoke({"input":"Tell me two things 1)Who are authors of paper Attention is all you need ? and 2) What is attention ?"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `Pdf_search` with `{'query': 'authors of the paper Attention is all you need'}`


Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this

In [ ]:
response=agent_executor.invoke({"input":"Tell me two things 1)Who are authors of paper Attention is all you need ? and 2) What is attention ?"})
print(response)